In [82]:
import pandas as pd
import os
from datetime import datetime
historical_data_file = os.getcwd() + '/historicalData.csv'
source_price_data_file = os.getcwd() + '/simResults-2019.csv'
#df0=pd.read_csv(historical_data_file)
print ('loading {}'.format(source_price_data_file))
df = pd.read_csv(source_price_data_file)
print ('shape of source file {}'.format(df.shape))
print ('source data header:')
for i in df:
    print (i, end=' ')
daily_supply_charge = 0.924   # $ /day
general_usage_rates = 31.79  # cents /kWh
solar_feed_in = 0.111    # $ /kWh exported

price_file = os.getcwd() + '/Price_data_2018.csv'
print ('\nloading {}'.format(price_file))

priceData = pd.read_csv(price_file)

#model 1: (old model)
#1. From the historical data, we can compute the following variables:
#AQmax=maximum(Main grid price) 
AQmax = max(priceData.spot_price)      # get 2.4645200000000003


print ('AQmax = ', AQmax)
user_id_max = max(df.user_id)             #the numbr of user_id in the priceData 14? 500?
print ('max user id number %s'%user_id_max)

number_Time_step = int(len(df) / user_id_max)

print ('number_Time_step %s'%number_Time_step)

netCostprosumer = []
for i in range (0, number_Time_step):   
    df_ = df[i*user_id_max : (i+1)*user_id_max]     
    netCostprosumer.append(sum(df_.purchase) - sum(df_.feed_in))
netCostMin = min(netCostprosumer)       
netCostMax = max(netCostprosumer)        
print ('netCostMax {},netCostMin {}'.format(netCostMax, netCostMin))  
#NetcostMin=minimum(∑_prosumers▒〖purchased 〗-∑_prosumers▒〖feedin 〗)



#2. With the real-time feed-in and purchased of 500 prosumers, the Feed-in price is computed as:
#esp model:
#feedInPrice = 
#0.01+0.6(CurrentNetCost-min(-6811779.069,CurrentNetCost))/(max(1488583.836,CurrentNetCost)-min(-6811779.069,CurrentNetCost))*(0.1-0.01)
print ('creating feed in price...')
feedInPrice = []
feedInTarrif = 0.1116
for i in range (0, number_Time_step):
    df_ = df[i*user_id_max: (i +1)* user_id_max]
    currentNetCost = sum(df_.purchase) - sum(df_.feed_in)
    Q = feedInTarrif + 0.05*(currentNetCost - min(netCostMin,currentNetCost)) / (max(netCostMax, currentNetCost) - min(netCostMin, currentNetCost)) * (AQmax-feedInTarrif)    
    feedInPrice.append(Q)   


#	Function / Method 2 – to calculate the real-time cost of a specified prosumer (Netcost_prosumer) if the price plan is implemented;

#Netcost_prosumer=retail_price*purchased_prosumer-Q*feedin_prosumer
#	Function / Method 3 – to calculate the real-time benefit of the DSP (Bene_DSP) if the price plan is implemented.
#Bene_DSP=1.3*(selling price*∑_prosumers▒〖purchased 〗-(main grid price*maximum(0,∑_prosumers▒〖purchased 〗-∑_prosumers▒〖feedin 〗)+Q*∑_prosumers▒〖feedin 〗))
print ('creating sum feed in ...')
sumPurchased=[]
sumFeedIn=[]
BeneDSP = []
for i in range (0,number_Time_step):
    df_ = df[i*user_id_max: (i +1)* user_id_max]
    #df_ = df.loc[df['user_id'] == id_number]   
    sumPurchased.append(sum(df_.purchase))
    sumFeedIn.append(sum(df_.feed_in))

print(len(priceData.retail_price), len(sumPurchased), len(priceData.spot_price),len(sumFeedIn),len(feedInPrice))

currentTimeMax=len(sumPurchased)

print ("calculating BeneDSP")
BeneDSP=[]
for i in range (0,number_Time_step):
    temp = 1.3*(priceData.retail_price[i]*sumPurchased[i] - (priceData.spot_price[i]*(max(0,sumPurchased[i]-sumFeedIn[i])+feedInPrice[i]*sumFeedIn[i])))
    BeneDSP.append(temp)
                

print ('len of BeneDSP %s'%len(BeneDSP))

#3. With the real-time selling price, the feed-in and purchased of a prosumer and the computed feed-in price Q, the real-time cost of the prosumer is computed as:
#Netcost_prosumer=selling price*purchased_prosumer-Q*feedin_prosumer

loading C:\Users\Administrator\Desktop\new_scaleup_closeUP_project\data_model_16-1-2019/simResults-2019.csv
shape of source file (12246480, 11)
source data header:
user_id date_time purchase feed_in consumption pv_generation DSP latitude longitude subtown user_price 
loading C:\Users\Administrator\Desktop\new_scaleup_closeUP_project\data_model_16-1-2019/Price_data_2018.csv
AQmax =  2.4645200000000003
max user id number 699
number_Time_step 17520
netCostMax 1286.1268724612428,netCostMin -8057.530914425463
creating feed in price...
creating sum feed in ...
17520 17520 17520 17520 17520
calculating BeneDSP
len of BeneDSP 17520


In [89]:
dt = []
for date_time in df.loc[df['user_id'] == 1].date_time.values:
    dt.append(datetime.strptime(date_time, '%Y/%m/%d/%H:%M'))
NetCost = []
table2= {}
time_step = []
temp=[]
#NetCost["customer_id"] =[]
#NetCost["NetElecCost"] =[]
#resultTable: timeStep, BeneDSP, feedInPrice, priceData.retail_price, priceData.spot_price
custNetCost = {}

print ("Calculating NetCost...")
for id_number in range (1,user_id_max+1):
    df_ = df.loc[df['user_id'] == id_number]
   
    #print(len(priceData.retail_price ), len(df_.purchase),len(feedInPrice),len(df_.feed_in) )
    temp = priceData.retail_price * df_.purchase.values - feedInPrice* df_.feed_in.values
    
    #valName='Netcost' + str(id_number)
    NetCost.append(temp.values)
    ###time_step.append(dt)
    #resultTable <- temp # temp means custNetCost
    print(len(priceData.retail_price ), len(df_.purchase),len(feedInPrice),len(df_.feed_in), len(temp),len(dt))
    
   # NetCost.append(temp)

#every time point

 

table = {}
table = {'BeneDSP': BeneDSP, 'feedInPrice':feedInPrice, 'retail_price':priceData.retail_price, 'spot_price':priceData.spot_price}
table_df = pd.DataFrame(table)
table_df['timeStep']= pd.DataFrame(dt)
print ('table head:\n',table_df.head())

Calculating NetCost...
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520

17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 1752

17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 1752

17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
17520 17520 17520 17520 17520 17520
table head:
      BeneDSP  feedInPrice  retail_price  spot_price            timeStep
0  33.129871     0.215531        0.2213     0.09186 2018-01-01 00:00:00
1  29.913119     0.215239        0.2213     0.08883 2018-01-01 00:30:00
2  33.373052     0.215241        0.2213     0.07362 2018-01-01 01:00:00
3  39.365760     0.215597        0.2213     0.07149 2018-01-01 01:30:00
4  37.591596     0.215447        0.2213     0.06927 2018-01-01 02:00:00


In [92]:
len(NetCost)

699

In [96]:
for iD in range(1,len(NetCost)+1):
    valName = "cost" + str(iD)
    table_df[valName] = NetCost[iD-1]

In [98]:
table_df.tail()

,BeneDSP,feedInPrice,retail_price,spot_price,timeStep,cost1,cost2,cost3,cost4,cost5,...,cost690,cost691,cost692,cost693,cost694,cost695,cost696,cost697,cost698,cost699
17515,62.647481,0.217317,0.2213,0.07903,2018-12-31 21:30:00,0.105883,0.161885,0.051919,0.889291,0.069154,...,0.027474,0.015343,0.005928,0.092956,0.030867,0.029579,0.113474,0.004598,0.091025,0.040242
17516,52.549217,0.216601,0.2213,0.07789,2018-12-31 22:00:00,0.080610,0.157070,0.041745,0.541752,0.055603,...,0.035151,0.011507,0.012566,0.070769,0.027606,0.037843,0.110099,0.005785,0.148469,0.035991
17517,59.437430,0.217071,0.2213,0.07806,2018-12-31 22:30:00,0.103704,0.117953,0.040693,0.807518,0.054202,...,0.089695,0.017474,0.004979,0.091043,0.031302,0.096565,0.082680,0.006526,0.166144,0.040809
17518,55.448157,0.216631,0.2213,0.07125,2018-12-31 23:00:00,0.099783,0.106519,0.081035,0.449757,0.107936,...,0.108684,0.014064,0.012566,0.087600,0.022172,0.117009,0.074665,0.008751,0.114003,0.028906
17519,61.531477,0.216994,0.2213,0.07011,2018-12-31 23:30:00,0.076689,0.120360,0.029117,0.725744,0.038782,...,0.118381,0.011933,0.004505,0.067326,0.007173,0.127449,0.084367,0.012311,0.087490,0.009352


In [99]:
print ('saving data to csv...')
table_df.to_csv('resultTable.csv', encoding='utf-8', index = False)

print("data file: resultTable.csv created.")

saving data to csv...
data file: resultTable.csv created.


In [100]:
#NetCost
print ('ctrating customer Net Cost csv...')
netcostrow=[]
table3= []

for a in range(len(NetCost[1])):
    #dic ={'date_time':dt[a]}
    for b in range(len(NetCost)):
        
        dic = {'CostNet':NetCost[b][a], 'customer_id':b,'date_time':dt[a]}
        table3.append(dic)
        

In [101]:
tbbb = pd.DataFrame(table3)
tbbb.to_csv('customerNetCost.csv')

In [103]:
tbbb.tail()

,CostNet,customer_id,date_time
12246475,0.127449,694,2018-12-31 23:30:00
12246476,0.084367,695,2018-12-31 23:30:00
12246477,0.012311,696,2018-12-31 23:30:00
12246478,0.087490,697,2018-12-31 23:30:00
12246479,0.009352,698,2018-12-31 23:30:00


In [66]:
df['CustNetCost'] = tbbb['CostNet']
#df['Cust_test_id']= tbbb['customer_id']  +1                          

#del df['Cust_test_id']
del df['user_price']

In [69]:
df.to_csv('simResultCustCostNet.csv')

KeyboardInterrupt: 